In [1]:
## import dan config
import pandas as pd
import numpy as np
import duckdb

from sklearn.model_selection import train_test_split

pd.options.display.max_columns = None

In [2]:
## load raw dataset
credit_applications = pd.read_csv("../datasets/raw/loan_data_2015.csv")

## display
credit_applications

C:\Users\DELL\AppData\Local\Temp\ipykernel_76236\748985828.py:2: DtypeWarning: Columns (19,47,55) have mixed types. Specify dtype option on import or set low_memory=False.
  credit_applications = pd.read_csv("../datasets/raw/loan_data_2015.csv")


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,60516983,64537751,20000,20000,20000,36 months,12.29,667.06,C,C1,Accounting Clerk,1 year,OWN,65000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,542xx,WI,20.72,0,Sep-00,1,NaN,NaN,25,0,31578,77.0,42,w,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,NaN,0.00,NaN,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,52303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41000,NaN,NaN,NaN
1,60187139,64163931,11000,11000,11000,36 months,12.69,369.00,C,C2,Accounts Payable Lead,7 years,MORTGAGE,40000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,235xx,VA,24.57,0,Sep-02,0,36.0,80.0,13,1,5084,38.8,41,w,0.00,0.00,10043.49,10043.49,9942.67,100.81,0.0,0.0,0.0,Oct-15,10059.00,NaN,Jan-16,0,79.0,1,INDIVIDUAL,NaN,NaN,NaN,0,332,175731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13100,NaN,NaN,NaN
2,60356453,64333218,7000,7000,7000,36 months,9.99,225.84,B,B3,Nurse,6 years,MORTGAGE,32000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,350xx,AL,32.41,0,Feb-06,1,NaN,NaN,18,0,12070,74.0,36,f,0.00,0.00,221.96,221.96,167.56,54.40,0.0,0.0,0.0,Oct-15,225.84,NaN,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,202012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16300,NaN,NaN,NaN
3,59955769,63900496,10000,10000,10000,36 months,10.99,327.34,B,B4,Service Manager,10+ years,MORTGAGE,48000.0,Source Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,credit_card,Credit card refinancing,483xx,MI,30.98,0,Oct-99,2,NaN,NaN,18,0,22950,66.0,41,f,0.00,0.00,315.13,315.13,235.76,79.37,0.0,0.0,0.0,Oct-15,327.34,NaN,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,108235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34750,NaN,NaN,NaN
4,58703693,62544456,9550,9550,9550,36 months,19.99,354.87,E,E4,NaN,NaN,RENT,32376.0,Verified,Sep-15,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,546xx,WI,32.54,0,Nov-99,3,69.0,NaN,9,0,4172,29.6,26,w,0.00,0.00,333.66,333.66,195.78,137.88,0.0,0.0,0.0,Oct-15,354.87,NaN,Jan-16,0,69.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,45492,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14100,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421089,37257620,40030388,25000,25000,25000,36 months,20.99,941.75,E,E4,shop foreman,10+ years,MORTGAGE,55000.0,Verified,Jan-15,Late (31-120 days),n,https://www.lendingclub.com/browse/loanDetail....,NaN,debt_consolidation,Debt consolidation,380xx,TN,31.22,0,Oct-99,1,NaN,NaN,13,0,22438,63.2,18,f,20128.91,20128.91,8417.44,8417.44,4871.09,3546.35,0.0,0.0,0.0,Oct-15,941.75,Feb-16,Jan-16,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,167189,NaN,NaN,NaN,NaN,NaN,N

In [3]:
## karena hanya eksperimen, sampling aja pakai 25% random dari data yg ada
sample_credit_applications = credit_applications.sample(frac=0.25, replace=True, random_state=333).reset_index(drop=True)

Initial exploration

In [4]:
sample_credit_applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105274 entries, 0 to 105273
Data columns (total 74 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           105274 non-null  int64  
 1   member_id                    105274 non-null  int64  
 2   loan_amnt                    105274 non-null  int64  
 3   funded_amnt                  105274 non-null  int64  
 4   funded_amnt_inv              105274 non-null  int64  
 5   term                         105274 non-null  object 
 6   int_rate                     105274 non-null  float64
 7   installment                  105274 non-null  float64
 8   grade                        105274 non-null  object 
 9   sub_grade                    105274 non-null  object 
 10  emp_title                    99312 non-null   object 
 11  emp_length                   99332 non-null   object 
 12  home_ownership               105274 non-null  object 
 13 

In [5]:
## hitung percentage null dari tiap kolom, filter yg percentagenya di atas 80%
na_values = sample_credit_applications.isnull().mean()
na_values[na_values>0.8].sort_values(ascending=False)

desc                         0.999782
dti_joint                    0.998746
annual_inc_joint             0.998727
verification_status_joint    0.998727
il_util                      0.955820
mths_since_rcnt_il           0.950909
open_il_6m                   0.949665
open_il_12m                  0.949665
open_il_24m                  0.949665
total_bal_il                 0.949665
open_acc_6m                  0.949665
open_rv_12m                  0.949665
open_rv_24m                  0.949665
max_bal_bc                   0.949665
all_util                     0.949665
inq_fi                       0.949665
total_cu_tl                  0.949665
inq_last_12m                 0.949665
mths_since_last_record       0.822995
dtype: float64

In [6]:
## drop kolom yg punya null lebih dari 80%
sample_credit_applications.dropna(thresh=sample_credit_applications.shape[0]*0.2, 
                           axis=1, inplace=True)


display(sample_credit_applications.head(3), sample_credit_applications.shape)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,41102320,43978216,13300,13300,13200,36 months,6.68,408.73,A,A3,Senior Maintenance Mechanic,5 years,MORTGAGE,75000.0,Verified,Feb-15,Current,n,https://www.lendingclub.com/browse/loanDetail....,debt_consolidation,Debt consolidation,840xx,UT,28.79,1,May-96,1,15.0,19,0,93073,88.5,36,w,9867.97,9793.78,4091.84,4061.07,3432.03,659.81,0.0,0.0,0.0,Dec-15,418.21,Jan-16,Jan-16,0,NaN,1,INDIVIDUAL,0,71,259473,99600
1,57990928,61794656,8000,8000,8000,36 months,13.33,270.83,C,C3,licensed practical nurse,3 years,MORTGAGE,35000.0,Source Verified,Aug-15,Current,n,https://www.lendingclub.com/browse/loanDetail....,home_improvement,Home improvement,422xx,KY,3.98,3,Sep-04,2,17.0,15,0,2178,41.9,28,w,7069.74,7069.74,1342.30,1342.30,930.26,412.04,0.0,0.0,0.0,Jan-16,270.83,Feb-16,Jan-16,0,17.0,1,INDIVIDUAL,0,0,25980,5200
2,45794657,48902376,10000,10000,10000,36 months,7.89,312.86,A,A5,Roofer,10+ years,MORTGAGE,42000.0,Source Verified,Apr-15,Current,n,https://www.lendingclub.com/browse/loanDetail....,credit_card,Credit card refinancing,957xx,CA,5.97,0,May-03,1,59.0,13,1,5448,17.2,39,w,7373.43,7373.43,3141.23,3141.23,2626.57,514.66,0.0,0.0,0.0,Jan-16,350.00,Feb-16,Jan-16,0,59.0,1,INDIVIDUAL,0,120,5448,31700


(105274, 55)

In [7]:
## drop kolom yg punya null lebih dari 80%
sample_credit_applications.dropna(thresh=sample_credit_applications.shape[0]*0.2, 
                           axis=1, inplace=True)

display(sample_credit_applications.head(3), sample_credit_applications.shape)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,41102320,43978216,13300,13300,13200,36 months,6.68,408.73,A,A3,Senior Maintenance Mechanic,5 years,MORTGAGE,75000.0,Verified,Feb-15,Current,n,https://www.lendingclub.com/browse/loanDetail....,debt_consolidation,Debt consolidation,840xx,UT,28.79,1,May-96,1,15.0,19,0,93073,88.5,36,w,9867.97,9793.78,4091.84,4061.07,3432.03,659.81,0.0,0.0,0.0,Dec-15,418.21,Jan-16,Jan-16,0,NaN,1,INDIVIDUAL,0,71,259473,99600
1,57990928,61794656,8000,8000,8000,36 months,13.33,270.83,C,C3,licensed practical nurse,3 years,MORTGAGE,35000.0,Source Verified,Aug-15,Current,n,https://www.lendingclub.com/browse/loanDetail....,home_improvement,Home improvement,422xx,KY,3.98,3,Sep-04,2,17.0,15,0,2178,41.9,28,w,7069.74,7069.74,1342.30,1342.30,930.26,412.04,0.0,0.0,0.0,Jan-16,270.83,Feb-16,Jan-16,0,17.0,1,INDIVIDUAL,0,0,25980,5200
2,45794657,48902376,10000,10000,10000,36 months,7.89,312.86,A,A5,Roofer,10+ years,MORTGAGE,42000.0,Source Verified,Apr-15,Current,n,https://www.lendingclub.com/browse/loanDetail....,credit_card,Credit card refinancing,957xx,CA,5.97,0,May-03,1,59.0,13,1,5448,17.2,39,w,7373.43,7373.43,3141.23,3141.23,2626.57,514.66,0.0,0.0,0.0,Jan-16,350.00,Feb-16,Jan-16,0,59.0,1,INDIVIDUAL,0,120,5448,31700


(105274, 55)

In [8]:
sample_credit_applications["emp_title"].nunique()

35477

In [9]:
## drop redundant dan forward-looking kolom
## redundant seperti id dan member_id asumsinya adalah salah satunya aja cukup
## forward-looking seperti recoveries, collection_recovery_fee, dll
## drop next_pymnt_d karena informasinya adalah tanggal bayar berikutnya

sample_credit_applications.drop(columns=['member_id', 'sub_grade', 'emp_title', 'url', 'title', 'zip_code', 'next_pymnt_d',
                          'recoveries', 'collection_recovery_fee', 'total_rec_prncp', 'total_rec_late_fee'], 
                          inplace=True)

In [10]:
display(sample_credit_applications.head(3), sample_credit_applications.shape)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,41102320,13300,13300,13200,36 months,6.68,408.73,A,5 years,MORTGAGE,75000.0,Verified,Feb-15,Current,n,debt_consolidation,UT,28.79,1,May-96,1,15.0,19,0,93073,88.5,36,w,9867.97,9793.78,4091.84,4061.07,659.81,Dec-15,418.21,Jan-16,0,NaN,1,INDIVIDUAL,0,71,259473,99600
1,57990928,8000,8000,8000,36 months,13.33,270.83,C,3 years,MORTGAGE,35000.0,Source Verified,Aug-15,Current,n,home_improvement,KY,3.98,3,Sep-04,2,17.0,15,0,2178,41.9,28,w,7069.74,7069.74,1342.30,1342.30,412.04,Jan-16,270.83,Jan-16,0,17.0,1,INDIVIDUAL,0,0,25980,5200
2,45794657,10000,10000,10000,36 months,7.89,312.86,A,10+ years,MORTGAGE,42000.0,Source Verified,Apr-15,Current,n,credit_card,CA,5.97,0,May-03,1,59.0,13,1,5448,17.2,39,w,7373.43,7373.43,3141.23,3141.23,514.66,Jan-16,350.00,Jan-16,0,59.0,1,INDIVIDUAL,0,120,5448,31700


(105274, 44)

**Flagging target**

In [11]:
## explore karakteristik dari kolom loan_status
sample_credit_applications['loan_status'].value_counts(normalize=True)

loan_status
Current               0.898398
Fully Paid            0.054372
Issued                0.019587
Late (31-120 days)    0.010601
In Grace Period       0.007191
Charged Off           0.006431
Late (16-30 days)     0.002517
Default               0.000902
Name: proportion, dtype: float64

In [12]:
## flagging target sebagai kolom baru based on loan_status
sample_credit_applications["credit_event"] = np.where(
    sample_credit_applications.loc[:, 'loan_status'].isin([
        "Charged Off", "Default", "Late (31-120 days)"
    ]), 1, 0
)

## drop loan_status
sample_credit_applications.drop(columns=["loan_status"], inplace=True)

sample_credit_applications.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,credit_event
0,41102320,13300,13300,13200,36 months,6.68,408.73,A,5 years,MORTGAGE,75000.0,Verified,Feb-15,n,debt_consolidation,UT,28.79,1,May-96,1,15.0,19,0,93073,88.5,36,w,9867.97,9793.78,4091.84,4061.07,659.81,Dec-15,418.21,Jan-16,0,NaN,1,INDIVIDUAL,0,71,259473,99600,0
1,57990928,8000,8000,8000,36 months,13.33,270.83,C,3 years,MORTGAGE,35000.0,Source Verified,Aug-15,n,home_improvement,KY,3.98,3,Sep-04,2,17.0,15,0,2178,41.9,28,w,7069.74,7069.74,1342.30,1342.30,412.04,Jan-16,270.83,Jan-16,0,17.0,1,INDIVIDUAL,0,0,25980,5200,0
2,45794657,10000,10000,10000,36 months,7.89,312.86,A,10+ years,MORTGAGE,42000.0,Source Verified,Apr-15,n,credit_card,CA,5.97,0,May-03,1,59.0,13,1,5448,17.2,39,w,7373.43,7373.43,3141.23,3141.23,514.66,Jan-16,350.00,Jan-16,0,59.0,1,INDIVIDUAL,0,120,5448,31700,0
3,54404871,11000,11000,11000,36 months,11.53,362.90,B,8 years,MORTGAGE,62000.0,Source Verified,Jul-15,n,debt_consolidation,CA,30.27,2,Jun-95,0,17.0,36,0,13906,50.9,69,w,0.00,0.00,11593.09,11593.09,593.09,Jan-16,9782.12,Jan-16,0,NaN,1,INDIVIDUAL,0,0,371838,27300,0
4,58574135,2200,2200,2200,36 months,6.24,67.17,A,6 years,MORTGAGE,103000.0,Source Verified,Aug-15,n,debt_consolidation,GA,28.53,0,May-94,0,NaN,20,0,18751,60.1,34,w,1975.34,1975.34,267.92,267.92,43.26,Jan-16,67.17,Jan-16,0,NaN,1,INDIVIDUAL,0,0,453718,31200,0


Split menjadi train-test (80%) dan validation (20%), dimana validation tidak akan digunakan atau disentuh proses validasi nantinya.

In [13]:
## split 20% for validation, 80% untuk train-test
train_test_set, validation_set = train_test_split(sample_credit_applications, test_size=0.2, random_state=777)

## cek hasil split
train_shape = train_test_set.shape
valid_shape = validation_set.shape

print("train-test shape: ", train_shape)
print("validation shape: ", valid_shape)

train-test shape:  (84219, 44)
validation shape:  (21055, 44)


In [14]:
display(train_test_set.head(3), validation_set.head(3))

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,credit_event
95775,57538238,8000,8000,8000,36 months,12.69,268.36,C,10+ years,OWN,100000.0,Source Verified,Aug-15,n,home_improvement,NC,3.73,0,Aug-08,0,24.0,5,0,17047,48.0,11,w,7061.56,7061.56,1330.52,1330.52,392.08,Jan-16,268.36,Jan-16,0,24.0,1,INDIVIDUAL,0,0,84258,35500,0
4065,51986192,25000,25000,25000,60 months,9.17,521.03,B,10+ years,MORTGAGE,85000.0,Source Verified,Jun-15,n,credit_card,OR,13.79,0,Jan-98,0,28.0,7,0,22629,69.4,23,w,22636.45,22636.45,3634.47,3634.47,1270.92,Jan-16,521.03,Jan-16,0,28.0,1,INDIVIDUAL,0,0,331816,32600,0
14327,61903317,7500,7500,7500,36 months,14.65,258.71,C,9 years,MORTGAGE,47000.0,Verified,Oct-15,n,debt_consolidation,CA,21.04,2,Feb-03,0,3.0,12,1,10635,41.7,17,f,6992.41,6992.41,770.03,770.03,262.44,Jan-16,258.71,Jan-16,0,NaN,1,INDIVIDUAL,0,0,359334,25500,0


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,credit_event
43626,59132268,20000,20000,20000,36 months,12.69,670.90,C,9 years,MORTGAGE,130000.0,Not Verified,Sep-15,n,debt_consolidation,GA,23.42,2,Jan-03,1,13.0,16,0,12386,43.9,44,w,18133.05,18133.05,2648.35,2648.35,781.40,Jan-16,670.90,Jan-16,0,NaN,1,INDIVIDUAL,0,0,307968,28200,0
14148,58101208,10000,10000,10000,60 months,19.99,264.89,E,10+ years,MORTGAGE,150000.0,Source Verified,Sep-15,n,debt_consolidation,TN,9.57,0,Feb-03,5,NaN,14,1,12657,25.1,26,f,9596.84,9596.84,1059.55,1059.55,656.39,Jan-16,264.89,Jan-16,0,NaN,1,INDIVIDUAL,0,0,170223,50400,0
51011,43946294,25325,25325,25325,36 months,18.84,926.27,E,5 years,MORTGAGE,142000.0,Verified,Mar-15,n,credit_card,MD,12.83,0,Mar-09,1,NaN,14,0,13990,79.9,22,w,20256.98,20256.98,8309.92,8309.92,3241.90,Dec-15,926.27,Jan-16,0,NaN,1,INDIVIDUAL,0,0,362049,17500,0


dari validation, kita akan ambil sekitar 5000an rows lagi untuk scenario scoring, dimana data untuk scoring ini nantinya tidak punya label target. in-practice, data ini mestinya gak diambil dari populasi validation sih, cuma bisa kita akalin sementara buat skenario sederhana.

In [15]:
to_validation_set, to_scoring_set = train_test_split(validation_set, test_size=0.25, random_state=777)

## cek hasil split
valid_shape = to_validation_set.shape
scoring_shape = to_scoring_set.shape

print("to validation shape: ", valid_shape)
print("to scoring shape: ", scoring_shape)


to validation shape:  (15791, 44)
to scoring shape:  (5264, 44)


In [17]:
to_scoring_set.drop(columns=["credit_event"], inplace=True)

to_scoring_set.head(3)

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
83520,65616745,10000,10000,10000,36 months,5.32,301.15,A,3 years,MORTGAGE,78000.0,Not Verified,Nov-15,n,credit_card,MD,13.80,0,Feb-99,0,35.0,10,0,17253,51.8,15,w,9743.18,9743.18,295.24,295.24,38.42,Dec-15,301.15,Jan-16,0,NaN,1,INDIVIDUAL,0,0,332536,33300
36370,55797208,25000,25000,25000,36 months,5.32,752.87,A,3 years,MORTGAGE,98000.0,Not Verified,Jul-15,n,debt_consolidation,TN,20.01,0,Mar-97,0,43.0,13,0,31014,48.8,30,w,21761.24,21761.24,3756.76,3756.76,518.00,Dec-15,752.87,Jan-16,0,NaN,1,INDIVIDUAL,0,0,333694,63600
16146,40391548,10000,10000,10000,36 months,11.44,329.48,B,10+ years,RENT,75000.0,Source Verified,Feb-15,n,debt_consolidation,CA,12.74,0,May-95,2,NaN,13,1,7028,40.4,39,w,7298.04,7298.04,3617.92,3617.92,915.96,Jan-16,329.48,Jan-16,0,NaN,1,INDIVIDUAL,0,0,14435,17400


Dump ke database

In [18]:
con = duckdb.connect("../dagster-scorecard/data/staging/loan_data_2015.duckdb")

In [19]:
## dump train-test
con.sql("CREATE OR REPLACE TABLE train_test_set AS SELECT * FROM train_test_set")
con.sql("INSERT INTO train_test_set SELECT * FROM train_test_set")

In [20]:
## dump validation
con.sql("CREATE OR REPLACE TABLE validation_set AS SELECT * FROM to_validation_set")
con.sql("INSERT INTO validation_set SELECT * FROM to_validation_set")

In [21]:
## dump untuk scoring
con.sql("CREATE OR REPLACE TABLE new_applications_to_score AS SELECT * FROM to_scoring_set")
con.sql("INSERT INTO new_applications_to_score SELECT * FROM to_scoring_set")

Coba query

In [22]:
con.sql("SELECT id, credit_event, home_ownership, emp_length FROM train_test_set LIMIT 5")

┌──────────┬──────────────┬────────────────┬────────────┐
│    id    │ credit_event │ home_ownership │ emp_length │
│  int64   │    int32     │    varchar     │  varchar   │
├──────────┼──────────────┼────────────────┼────────────┤
│ 57538238 │            0 │ OWN            │ 10+ years  │
│ 51986192 │            0 │ MORTGAGE       │ 10+ years  │
│ 61903317 │            0 │ MORTGAGE       │ 9 years    │
│ 66625630 │            0 │ RENT           │ NULL       │
│ 67288208 │            0 │ OWN            │ 10+ years  │
└──────────┴──────────────┴────────────────┴────────────┘

In [23]:
con.sql("SELECT id, credit_event, home_ownership, emp_length FROM validation_set LIMIT 5")

┌──────────┬──────────────┬────────────────┬────────────┐
│    id    │ credit_event │ home_ownership │ emp_length │
│  int64   │    int32     │    varchar     │  varchar   │
├──────────┼──────────────┼────────────────┼────────────┤
│ 43285674 │            0 │ RENT           │ 10+ years  │
│ 40257226 │            1 │ MORTGAGE       │ 7 years    │
│ 63335678 │            0 │ RENT           │ 1 year     │
│ 59553280 │            0 │ MORTGAGE       │ 10+ years  │
│ 38608497 │            0 │ MORTGAGE       │ 6 years    │
└──────────┴──────────────┴────────────────┴────────────┘

In [25]:
con.sql("SELECT id, home_ownership, emp_length FROM new_applications_to_score LIMIT 5")

┌──────────┬────────────────┬────────────┐
│    id    │ home_ownership │ emp_length │
│  int64   │    varchar     │  varchar   │
├──────────┼────────────────┼────────────┤
│ 65616745 │ MORTGAGE       │ 3 years    │
│ 55797208 │ MORTGAGE       │ 3 years    │
│ 40391548 │ RENT           │ 10+ years  │
│ 40371290 │ RENT           │ 10+ years  │
│ 55957703 │ RENT           │ 4 years    │
└──────────┴────────────────┴────────────┘

In [26]:
con.close()

cleaning sandbox

In [48]:
con = duckdb.connect("../dagster-scorecard/data/staging/loan_data_2015.duckdb")
query = """SELECT * FROM train_test_set"""

df = con.execute(query).fetch_df()
df.drop_duplicates(subset=['id'], keep='first', inplace=True, ignore_index=True)


df

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,credit_event
0,55320288,11000,11000,11000,36 months,9.99,354.89,B,< 1 year,RENT,31300.0,Not Verified,Jul-15,n,credit_card,CO,25.50,0,Oct-96,1,NaN,10,0,7959,44.0,14,f,9386.86,9386.86,2123.24,2123.24,510.10,Jan-16,354.89,Jan-16,0,NaN,1,INDIVIDUAL,0,0,34398,18100,0
1,41930439,12600,12600,12600,36 months,12.69,422.67,C,10+ years,MORTGAGE,45000.0,Verified,Mar-15,n,debt_consolidation,WI,36.93,0,Apr-00,0,NaN,9,0,17157,81.3,15,f,9567.06,9567.06,4217.82,4217.82,1184.88,Jan-16,422.67,Jan-16,0,NaN,1,INDIVIDUAL,0,0,75847,21100,0
2,43801439,2000,2000,2000,36 months,8.18,62.84,B,10+ years,MORTGAGE,40000.0,Verified,Mar-15,n,major_purchase,FL,16.92,0,Mar-04,0,NaN,17,0,4377,17.1,21,f,1492.56,1492.56,626.58,626.58,119.14,Jan-16,62.84,Jan-16,0,NaN,1,INDIVIDUAL,0,0,57963,25600,0
3,598481,6000,6000,6000,36 months,11.53,197.95,B,9 years,RENT,42500.0,Source Verified,Oct-15,n,credit_card,OH,25.67,2,Nov-89,1,15.0,10,0,12372,41.1,32,w,5575.04,5575.04,586.16,586.16,161.20,Jan-16,197.95,Jan-16,0,15.0,1,INDIVIDUAL,0,0,13697,30100,0
4,55847195,10000,10000,10000,36 months,7.89,312.86,A,None,MORTGAGE,44000.0,Not Verified,Jul-15,n,credit_card,TX,34.74,1,Oct-04,2,8.0,12,1,7228,20.2,25,w,8492.75,8492.75,1872.78,1872.78,365.53,Jan-16,312.86,Jan-16,0,NaN,1,INDIVIDUAL,0,0,62183,35800,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50451,46446945,25000,25000,25000,60 months,10.99,543.44,B,8 years,MORTGAGE,59000.0,Source Verified,Apr-15,n,credit_card,MN,27.71,0,Dec-86,0,NaN,7,0,72045,79.6,12,w,22402.01,22402.01,4332.26,4332.26,1734.27,Jan-16,543.44,Jan-16,0,NaN,1,INDIVIDUAL,0,0,72045,103100,0
50452,60920737,2000,2000,2000,36 months,12.29,66.71,C,2 years,MORTGAGE,80000.0,Not Verified,Sep-15,n,credit_card,CA,22.45,1,Jan-82,0,6.0,17,1,9339,44.7,26,w,1859.89,1859.89,198.76,198.76,58.65,Dec-15,66.71,Jan-16,0,NaN,1,INDIVIDUAL,0,0,59743,20900,0
50453,60762054,8000,8000,8000,36 months,14.65,275.96,C,1 year,RENT,37000.0,Source Verified,Sep-15,n,debt_consolidation,MI,28.58,4,Aug-01,1,12.0,20,0,3382,76.9,50,f,7458.56,7458.56,844.16,844.16,302.72,Jan-16,275.96,Jan-16,0,14.0,1,INDIVIDUAL,0,0,110574,4400,0
50454,59679287,21000,21000,21000,36 months,12.29,700.42,C,7 years,MORTGAGE,250000.0,Source Verified,Oct-15,n,credit_card,IL,7.82,0,Jun-98,2,62.0,16,0,22151,79.4,37,w,19529.01,19529.01,2172.94,2172.94,701.95,Jan-16,700.42,Jan-16,0,NaN,1,INDIVIDUAL,0,0,174012,27900,0


cleaning kolom time-related

In [49]:
## cleaning emp_length
def emp_length_converter(dataframe, column):
    dataframe[column] = dataframe[column].replace({
        r"\+ years": "",
        r"< 1 year": "0",
        r" years?": ""
    }, regex=True)
    
    dataframe[column] = pd.to_numeric(dataframe[column], errors='coerce').fillna(0).astype(int)

## apply
emp_length_converter(df, "emp_length")

## cek hasil
## cek statistiknya
display(df["emp_length"].describe(), df.head(2))

count    50456.000000
mean         5.693158
std          3.855279
min          0.000000
25%          2.000000
50%          6.000000
75%         10.000000
max         10.000000
Name: emp_length, dtype: float64

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,credit_event
0,55320288,11000,11000,11000,36 months,9.99,354.89,B,0,RENT,31300.0,Not Verified,Jul-15,n,credit_card,CO,25.50,0,Oct-96,1,NaN,10,0,7959,44.0,14,f,9386.86,9386.86,2123.24,2123.24,510.10,Jan-16,354.89,Jan-16,0,NaN,1,INDIVIDUAL,0,0,34398,18100,0
1,41930439,12600,12600,12600,36 months,12.69,422.67,C,10,MORTGAGE,45000.0,Verified,Mar-15,n,debt_consolidation,WI,36.93,0,Apr-00,0,NaN,9,0,17157,81.3,15,f,9567.06,9567.06,4217.82,4217.82,1184.88,Jan-16,422.67,Jan-16,0,NaN,1,INDIVIDUAL,0,0,75847,21100,0


In [50]:
## cek term
df["term"].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [51]:
## remove whitespace
## coba keep sbg categorical dulu aja
df["term"] = df["term"].str.lstrip()

handle date-related columns, hitung time difference

In [52]:
## cek missing rate
na_values = df.isnull().sum()
na_values

id                                 0
loan_amnt                          0
funded_amnt                        0
funded_amnt_inv                    0
term                               0
int_rate                           0
installment                        0
grade                              0
emp_length                         0
home_ownership                     0
annual_inc                         0
verification_status                0
issue_d                            0
pymnt_plan                         0
purpose                            0
addr_state                         0
dti                                0
delinq_2yrs                        0
earliest_cr_line                   0
inq_last_6mths                     0
mths_since_last_delinq         24361
open_acc                           0
pub_rec                            0
revol_bal                          0
revol_util                        22
total_acc                          0
initial_list_status                0
o

In [53]:
## last_pymnt_d dan last_credit_pull_d punya missing values, padahal kita butuh convert dan hitung time difference mereka
## karena missingnya last_pymnt_d kurang dari 5%, rasanya masih save untuk didrop
## drop rows 
df.dropna(subset=["last_pymnt_d", "last_credit_pull_d"], inplace=True)

In [54]:
def convert_and_calc_moths(df):
    ## define kolom
    date_columns = ['earliest_cr_line', 'issue_d', 'last_pymnt_d', 'last_credit_pull_d']
    
    # convert ke datetime
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], format='%b-%y', errors='coerce')
    
    ## hitung time difference antara issue date dgn earliest credit line
    df['months_since_earliest_cr_line'] = (df['issue_d'].dt.year - df['earliest_cr_line'].dt.year) * 12 + \
                                          (df['issue_d'].dt.month - df['earliest_cr_line'].dt.month)
    
    ## hitung time difference antara last payment date dng issue date
    df['months_since_issue_d_last_pymnt_d'] = (df['last_pymnt_d'].dt.year - df['issue_d'].dt.year) * 12 + \
                                              (df['last_pymnt_d'].dt.month - df['issue_d'].dt.month)
    
    ## hitung time difference antara last credit pull date and last payment date
    df['months_since_last_pymnt_d_last_credit_pull_d'] = (df['last_credit_pull_d'].dt.year - df['last_pymnt_d'].dt.year) * 12 + \
                                                         (df['last_credit_pull_d'].dt.month - df['last_pymnt_d'].dt.month)
    
    ## hitung time difference antara issue date and last credit pull date
    df['months_since_issue_d_last_credit_pull_d'] = (df['last_credit_pull_d'].dt.year - df['issue_d'].dt.year) * 12 + \
                                                    (df['last_credit_pull_d'].dt.month - df['issue_d'].dt.month)

    ## drop kolom original
    df.drop(columns=date_columns, inplace=True)
    
    return df


In [55]:
## apply function
df = convert_and_calc_moths(df)

## cek result
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_int,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,credit_event,months_since_earliest_cr_line,months_since_issue_d_last_pymnt_d,months_since_last_pymnt_d_last_credit_pull_d,months_since_issue_d_last_credit_pull_d
0,55320288,11000,11000,11000,36 months,9.99,354.89,B,0,RENT,31300.0,Not Verified,n,credit_card,CO,25.50,0,1,NaN,10,0,7959,44.0,14,f,9386.86,9386.86,2123.24,2123.24,510.10,354.89,0,NaN,1,INDIVIDUAL,0,0,34398,18100,0,225,6,0,6
1,41930439,12600,12600,12600,36 months,12.69,422.67,C,10,MORTGAGE,45000.0,Verified,n,debt_consolidation,WI,36.93,0,0,NaN,9,0,17157,81.3,15,f,9567.06,9567.06,4217.82,4217.82,1184.88,422.67,0,NaN,1,INDIVIDUAL,0,0,75847,21100,0,179,10,0,10
2,43801439,2000,2000,2000,36 months,8.18,62.84,B,10,MORTGAGE,40000.0,Verified,n,major_purchase,FL,16.92,0,0,NaN,17,0,4377,17.1,21,f,1492.56,1492.56,626.58,626.58,119.14,62.84,0,NaN,1,INDIVIDUAL,0,0,57963,25600,0,132,10,0,10
3,598481,6000,6000,6000,36 months,11.53,197.95,B,9,RENT,42500.0,Source Verified,n,credit_card,OH,25.67,2,1,15.0,10,0,12372,41.1,32,w,5575.04,5575.04,586.16,586.16,161.20,197.95,0,15.0,1,INDIVIDUAL,0,0,13697,30100,0,311,3,0,3
4,55847195,10000,10000,10000,36 months,7.89,312.86,A,0,MORTGAGE,44000.0,Not Verified,n,credit_card,TX,34.74,1,2,8.0,12,1,7228,20.2,25,w,8492.75,8492.75,1872.78,1872.78,365.53,312.86,0,NaN,1,INDIVIDUAL,0,0,62183,35800,0,129,6,0,6


In [56]:
con.close()